In [87]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt


%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly import graph_objs as go
import cufflinks as cf

%qtconsole
#cf.go_offline()

init_notebook_mode()

## no more deprecation warnings please
import warnings
warnings.filterwarnings('ignore')

<h1> Data Munging! </h1>

All data is a static snapshot sourced from insideairbnb.com 

In [88]:
Listings = pd.read_csv('data/listings.csv')
Listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,6627449,Large 1 BDRM in Great location,13886510,Arlene,Bronx,City Island,40.849775,-73.786609,Entire home/apt,125,3,8,2016-01-05,1.12,1,337
1,7949480,City Island Sanctuary Sunny BR & BA,119445,Linda & Didier,Bronx,City Island,40.852054,-73.788680,Private room,68,1,1,2016-01-18,1.00,1,349
2,1886820,Quaint City Island Community.,9815788,Steve,Bronx,City Island,40.841144,-73.783052,Entire home/apt,300,7,0,NaN,NaN,1,365
3,5557381,Quaint City Island Home,28811542,Phoebe,Bronx,City Island,40.850024,-73.789328,Private room,49,1,41,2016-01-29,4.84,1,344
4,9019702,City Island Sugar Shack,46642112,Jody,Bronx,City Island,40.847917,-73.787731,Entire home/apt,200,1,0,NaN,NaN,1,365


In [89]:
Reviews = pd.read_csv('data/reviews.csv')
print Reviews.date.min()
print Reviews.date.max()

Reviews.date = pd.to_datetime(Reviews.date)
Reviews.head()

2008-10-06
2016-02-02


,listing_id,date
0,6987560,2015-06-29
1,6987560,2015-10-12
2,7652181,2015-08-17
3,7652181,2015-10-25
4,7652181,2015-10-30


In [90]:
## Confining the reviews to year 2015 + 

Reviews.set_index('date',inplace=True)
Reviews.sort(ascending=True,inplace=True)


start = Reviews.index.searchsorted(dt.datetime(2015, 1, 1)) ; #print Reviews.ix[start]
end   = Reviews.index.searchsorted(dt.datetime(2016, 1, 1)) ; #print Reviews.ix[end]

Year_Reviews = Reviews.ix[start:end]

print Year_Reviews.idxmin()
print Year_Reviews.idxmax()



listing_id   2015-01-10
dtype: datetime64[ns]
listing_id   2015-12-30
dtype: datetime64[ns]


In [91]:
Year_Reviews['counter'] = 1
reviews_in_period = Year_Reviews.groupby(['listing_id',pd.TimeGrouper('M')]).count()['counter']

reviews_per_month_period = pd.DataFrame(reviews_in_period.unstack().fillna(0).mean(axis=1))
reviews_per_month_period.columns = ['reviews_per_month_period']

reviews_per_month_period.fillna(0,inplace = True)

reviews_per_month_period.head()


,reviews_per_month_period
listing_id,
2515,1.500000
2534,0.083333
2539,0.083333
2595,0.416667
3330,0.666667


In [92]:
joined = Listings.merge(reviews_per_month_period, left_on='id',right_index=True, how='left')

joined.reviews_per_month_period.fillna(0,inplace=True)

<h1> The Revenue Model </h1>

I am using the InsideAirBnB "San Francisco" Revenue Model, found here http://insideairbnb.com/about.html

Key Assumptions, shared with SF Model: 
    * The average stay is 3 nights
    * Every other customer leaves a review i.e., review rate of 50%
    * The occupancy rate is capped at 70% i.e., each AirBnB can be occuped a max of 21 days a month
    
Key Assumptions, **not** shared with SF Model: 
    * Throw out all listings with nightly price > 2500. This number is somewhat arbitrary but prices
        higher than this do not seem believable for a 'nightly' rental, even for the most opulent apartment.
        They may be long-term listings (2500 a month seems reasonable) that are being classified incorrectly 
        by the scraper. 
        

    


In [93]:
joined = joined[joined.last_review >= start]  ## keep reviews in period 

## filtering out listings with price > 2500, not scientifically based but that seems like a high limit for believable
## listings
joined = joined[joined.price < 2501]

## 3 night avg stay, unless the minimum is higher
joined['avg_stay_nights'] = np.where(joined.minimum_nights > 3, joined.minimum_nights, 3) 


review_rate = .5 # sourced from insideAirbnb, conservative estimate

## capping occupany at 70% = 21 days
joined['occupancy_nights_month'] = joined.reviews_per_month_period * (1/review_rate) * joined.avg_stay_nights


joined['occupancy_nights_month'] = np.where(joined.occupancy_nights_month <= 21,joined.occupancy_nights_month,21 )

joined['annual_revenue'] = joined.occupancy_nights_month * joined.price * 12

joined.groupby('neighbourhood_group').mean()['occupancy_nights_month']

neighbourhood_group
Bronx            5.096284
Brooklyn         4.646566
Manhattan        4.894646
Queens           5.091754
Staten Island    4.693853
Name: occupancy_nights_month, dtype: float64

<h2> Active Listings </h2>

Just checking what percentage of listings actually have at least one review during 2015. 

In [94]:
joined['active_listings'] = joined.reviews_per_month_period.apply(lambda x: True if x > 0 else False)

pct_active = joined.groupby(['neighbourhood_group','active_listings'])['id'].count().unstack() \
            .apply(lambda x: x/ x.sum(),axis=1)
    
pct_active.iplot(kind='bar',barmode='stack',title='% of Listings Active in 2015')

In [95]:
## Supporting functions for visualization

def CountListings(pivot):
    return len(pivot) - pivot.isnull().sum()

    
def CreateMarketShare(df,col = 'neighbourhood_group' ,groupby='id', values = 'annual_revenue',filter_wholehome=True,filter_boro=False\
                    , normalize=True):
    if filter_wholehome:
        df = df[(df.room_type == 'Entire home/apt')]
    if filter_boro:
        df = df[(df.neighbourhood_group == filter_boro)]
        
    my_pivot = df.pivot_table(index=groupby,columns=col,values=values)
    if normalize:
        my_pivot = my_pivot.apply(lambda x: x/x.sum())
    
    print "Number of Listings Counted: "  
    print CountListings(my_pivot)
    return my_pivot


def PlotlyCDF(df,y_title='% of Revenue', x_title="% of Listings",chart_title='Whale Chart',use_matplot=False,just_data=False):
    
    data = [{
        'x': df[col].dropna().index.value_counts(normalize=True).cumsum(),
        'y': df[col].sort_values(ascending=False).cumsum().dropna(),
        'name': col
    }  for col in df.columns]
    
    if just_data:
        return data
   
    ## default to matplotlib if not connected to internet like I am right now on this flight ':(
    if use_matplot:
        plt.legend(df.columns.values)
        for d in data:
            plt.plot(d['x'],d['y'])
    
    else:
        layout=go.Layout(title= chart_title, xaxis={'title':x_title}, yaxis={'title': y_title})
        fig = go.Figure(data=data, layout=layout)
        iplot(fig)
    


<h1> Whale Charts! </h1>

My functions are plotting a kind of Cumulative Distribution that I've heard referred to as a "Whale Chart". They plot the percentage of a total variable of interest e.g., AirBnB revenue in NYC that is captured by a percentage of unique listings. They serve as a way to visualize market structure; a whale chart that is very steep near the left side of the picture shows a market dominated by a few large actors. A more egalitarian market structure would have a gentler sloping whale chart. 

AirBnB in New York City seems to be highly reliant on a small percentage of listings that drive listings revenue -- and by extension -- revenue for AirBnB itself. A casual glance confirms the top 20% of listings in NYC disproportionately capture about 60% of the total revenue in the city. 

While there is nothing inherently suprising about some users being more active than others, this does provide a new lense under which to view AirBnB's claims about their service. AirBnB proudly states that most of their users have only one listing, and this appears to be true. However, the company's revenue is heavily dependent on a small percentage of listings that make a lot of money. How do you make a lot of money off of a single apartment? By renting it out a lot. Sort of like an illegal hotel...



In [96]:
by_boro = CreateMarketShare(joined,filter_wholehome=False)
PlotlyCDF(by_boro,chart_title='Concentration of AirBnB Revenue (NYC)',use_matplot=False)

Number of Listings Counted: 
neighbourhood_group
Bronx              296
Brooklyn         10522
Manhattan        13945
Queens            2009
Staten Island      141
dtype: int64


In [97]:
by_host = CreateMarketShare(joined,filter_wholehome=True,groupby='host_id')
PlotlyCDF(by_host,use_matplot=False,chart_title='Concentration of AirBnB Revenue by Host (NYC)',x_title='% of Hosts')

Number of Listings Counted: 
neighbourhood_group
Bronx              76
Brooklyn         4849
Manhattan        8307
Queens            709
Staten Island      47
dtype: int64


In [98]:
scatter = joined[joined.neighbourhood_group=='Manhattan']

trace1 = go.Scattergl(
    x = scatter[(scatter.room_type == 'Entire home/apt')].occupancy_nights_month,
    y = scatter[(scatter.room_type == 'Entire home/apt')].annual_revenue,
    mode = 'markers',
    marker = dict(
        color = 'FFBAD2',
        line = dict(width = 1)
    )
)

trace2 = go.Scattergl(
    x = scatter[(scatter.room_type != 'Entire home/apt')].occupancy_nights_month,
    y = scatter[(scatter.room_type != 'Entire home/apt')].annual_revenue,
    mode = 'markers',
    marker = dict(
        color = 'FFBAD',
        line = dict(width = 1)
    )
)

data = [trace1,trace2]
iplot(data)

<h2> Neighborhood Level </h2>

Now I'm going to plot some selected whale charts for neighborhoods in Brooklyn to look for trends.

In [99]:
%%capture # suppressing output here

BK = CreateMarketShare(joined,col='neighbourhood',filter_boro='Brooklyn')

## limiting neighborhoods to places with > 30 listings 
count_bk = CountListings(BK) 
subset = count_bk[count_bk > 30].index

UsageError: unrecognized arguments: suppressing output here

In [100]:
PlotlyCDF(BK[subset],use_matplot=False, chart_title='Revenue By BK Neighborhood')

<h1> To Be Continued </h1>

This is a work in progress